In [ ]:
 A_inv = self.A_inv[pool_idx]
        b = self.b[pool_idx]

        n_pool = len(pool_idx)

        user = np.array([user] * n_pool)
        if self.context == 1:
            x = user
        else:
            x = np.hstack((user, dataset.features[pool_idx]))

        x = x.reshape(n_pool, self.n_features, 1)

        theta = A_inv @ b

        p = np.transpose(theta, (0, 2, 1)) @ x + self.alpha * np.sqrt(
            np.transpose(x, (0, 2, 1)) @ A_inv @ x
        )
        return np.argmax(p)

    def update(self, displayed, reward, user, pool_idx):

##  3.1 구조 개요

`LinUCB` 알고리즘은 각 **기사(arm)** 별로 선형 모델을 독립적으로 유지하면서, **사용자 컨텍스트**를 기반으로 가장 클릭 가능성이 높은 기사를 선택하는 **Upper Confidence Bound(UCB)** 기반의 컨텍스트 밴딧 알고리즘.

---

##  `__init__` 생성자

```python
def __init__(self, alpha, context="user"):
```

### 매개변수

* `alpha`: **탐험 탐색 균형**을 조절하는 파라미터 (UCB에서 불확실성 크기에 곱해짐)
* `context`: `"user"` 또는 `"both"` 중 선택. `"user"`면 사용자 특성만, `"both"`면 사용자+기사 특성 사용.

### 주요 초기화

```python
self.n_features = len(dataset.features[0])
```

* 기사 한 개의 feature 벡터 길이를 저장.

```python
if context == "both":
    self.n_features *= 2
```

* `"both"`일 경우 사용자 feature + 기사 feature 를 연결(concatenate)하므로 총 feature 수가 2배.

```python
self.A = np.array([np.identity(self.n_features)] * dataset.n_arms)
self.A_inv = np.array([np.identity(self.n_features)] * dataset.n_arms)
self.b = np.zeros((dataset.n_arms, self.n_features, 1))
```

* 각 arm(기사) 별로 다음의 선형 회귀 매트릭스를 유지:

  * `A`: $d \times d$ 행렬로 $A_a = \sum x x^\top$
  * `A_inv`: `A`의 역행렬 미리 저장
  * `b`: $d \times 1$ 벡터로 $b_a = \sum r x$

---

##  `choose_arm` 함수 – 어떤 arm을 고를래?

```python
def choose_arm(self, t, user, pool_idx):
```

### 입력

* `t`: 현재 시점
* `user`: 현재 사용자의 feature 벡터
* `pool_idx`: 추천 가능한 기사들의 인덱스 리스트

### 동작 요약

1. **사용자 feature 준비**

```python
user = np.array([user] * n_pool)
```

→ pool에 있는 모든 기사에 대해 사용자 feature를 복제

2. **feature vector `x` 생성**

```python
x = np.hstack((user, dataset.features[pool_idx]))
```

→ `"both"`일 경우, 사용자와 기사 feature를 연결

3. **UCB 계산**

```python
theta = A_inv @ b
p = θᵀ x + α * sqrt(xᵀ A⁻¹ x)
```

$$
\theta = A^{-1} b
$$

$$
p = \theta^\top x + \alpha \sqrt{x^\top A^{-1} x}
$$


각 arm의 보상 추정값 + 탐험 보정치를 계산

* $\hat{\theta}_a = A_a^{-1} b_a$

* $p_{t,a} = \hat{\theta}_a^\top x_{t,a} + \alpha \cdot \sqrt{x_{t,a}^\top A_a^{-1} x_{t,a}}$


4. **최고의 arm 선택**

```python
return np.argmax(p)
```

---

## 🔁 `update` 함수 – 클릭 여부로 모델 업데이트

```python
def update(self, displayed, reward, user, pool_idx):
```

### 입력

* `displayed`: `pool_idx`에서 몇 번째 arm이 선택됐는지
* `reward`: 클릭 여부 (1 or 0)
* `user`: 사용자 feature
* `pool_idx`: 추천 pool의 index

### 동작

1. 선택된 기사의 원래 인덱스 가져오기

```python
a = pool_idx[displayed]
```

2. feature 벡터 `x` 계산

```python
x = np.hstack((user, dataset.features[a]))
x = x.reshape((self.n_features, 1))
```

3. **모수 업데이트**

```python
self.A[a] += x @ x.T
self.b[a] += reward * x
self.A_inv[a] = np.linalg.inv(self.A[a])
```

이는 다음 식을 구현:
* $A_a \leftarrow A_a + x x^\top$
* $b_a \leftarrow b_a + r x$
* $\hat{\theta}_a = A_a^{-1} b_a$

---

## 💡 요약

| 구성요소                 | 설명                                   |
| -------------------- | ------------------------------------ |
| `A`, `b`             | 각 기사(arm)에 대한 선형 회귀 매개변수             |
| `choose_arm()`       | UCB 기반으로 현재 사용자에게 최적의 기사를 선택         |
| `update()`           | 클릭 피드백을 이용해 해당 기사(arm)의 모델 파라미터 업데이트 |
| `alpha`              | 탐험과 활용 균형 조절 파라미터                    |
| `"user"` vs `"both"` | 어떤 context 정보를 feature로 사용할지 결정      |

---

